In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *
import importlib, pipeline_helper

In [ ]:
importlib.reload(pipeline_helper)
from pipeline_helper import allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms

In [ ]:
import numpy as np
import numpy.linalg as la

In [ ]:
import pickle, gzip

### Initial Deployment

In [ ]:
name = 'sphere_cap_0.3'

In [ ]:
input_path = '../../data/{}.json'.format(name)
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
import mesh
target_mesh = mesh.Mesh(input_data['target_v'], input_data['target_f'])

In [ ]:
curr_um = pickle.load(gzip.open('sphere_cap_0.3_initialization_in_equilibrium_2022_01_16_18_08.pkl.gz', 'r'))

In [ ]:
target_height_multiplier = 5

#### Pin Rigid Motion



In [ ]:
use_pin = False

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

In [ ]:
rod_colors = get_color_field(curr_um, input_data)

In [ ]:
import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
# view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# view.update(scalarField = rod_colors)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

### Initialize Design Optimization

In [ ]:
import umbrella_optimization
import umbrella_optimization_finite_diff

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-3

In [ ]:
import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False

In [ ]:
from equilibrium_solve_analysis import EquilibriumSolveAnalysis
eqays = EquilibriumSolveAnalysis(curr_um)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = opt_opts, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
opt_opts.niter = 50

In [ ]:
results.success

In [ ]:
# pickle.dump(curr_um, gzip.open('{}_initialization_in_equilibrium_{}.pkl.gz'.format(name, time.strftime("%Y_%m_%d_%H_%M")), 'w'))

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

In [ ]:
optimizer.beta = 5 * 1e5
optimizer.gamma = 1
optimizer.eta = 0
optimizer.zeta = 1e1

In [ ]:
rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
single_rest_height_optimizer = umbrella_optimization.UmbrellaSingleRestHeightOptimization(rest_height_optimizer)

In [ ]:
import design_optimization_analysis

In [ ]:
doptays = design_optimization_analysis.DesignOptimizationAnalysis(rest_height_optimizer)

In [ ]:
import time
pipeline_helper.prev_time_stamp = time.time()

In [ ]:
cb = pipeline_helper.UmbrellaOptimizationCallback(rest_height_optimizer, view, True, False, 1, rod_colors, doptays.record)

In [ ]:
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 2, 1.0, 1e-2, cb, -1)

### Equilibrium solve with different parameters

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-2

In [ ]:
opt_opts.niter = 200

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = opt_opts, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
opt_opts.niter = 50

In [ ]:
results.success

In [ ]:
solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 2, 1.0, 1e-2, cb, -1)